# Demonstration Usage Fitting Procedures

In [ ]:
# Import FittingProcedures
from FittingProcedures import *
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
from scipy.ndimage import gaussian_filter1d
import os
from fit_dictionaries import *
from fit_tricks_to_compare_different_conditions import *
from Fit_spcific_cases import *
from FittingProcedures import *
from fit_plots import *
import os
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.ticker as ticker
import json
from typing import Dict, List, Tuple, Any, Optional

# Import functions from existing modules
from fit_tricks_to_compare_different_conditions import enrich_vector_to_length, from_feature_to_nin_bin_range, from_data_to_cut_distribution
from FittingProcedures import compare_exponential_power_law_from_xy, fit_expo
from FittingProcedures_average_days_functions import *


# Input Variables

In [ ]:
## ------------ SHARED VARIABLES ----------------- ##
# Important Folders
city_pro_dir = os.path.join(os.environ["WORKSPACE"],"city-pro") 
vars_dir = os.path.join(city_pro_dir, "vars")
# Config
dir_config_analysis_day = os.path.join(vars_dir,"config","config_days_bbox.json")
# N classes 
with open(os.path.join(dir_config_analysis_day),"r") as f:
    config_analysis_day = json.load(f)

# Name project-bbox
prefix_files = "bologna_mdt"
name_project_related_to_bbox = config_analysis_day["name_project"]


# Folders
config_dir = os.path.join(city_pro_dir,name_project_related_to_bbox)
fcm_dir = os.path.join(os.environ["WORKSPACE"],"city-pro","output",name_project_related_to_bbox)
PlotDir = os.path.join(fcm_dir, "plots")



# Parameters Days - Classes
StrDates = config_analysis_day["str_dates"]
num_tm = config_analysis_day["num_tm"] + 1
labels = [str(i) for i in range(num_tm)]
Classes = [int(i) for i in range(num_tm)]

# Variables of Interest
Features = ["time_hours", "lenght_km"]

# 
lon_min,lon_max,lat_min,lat_max = config_analysis_day["lon_min"], config_analysis_day["lon_max"], config_analysis_day["lat_min"], config_analysis_day["lat_max"]

# Filters 
filter_lenght_km = 10
filter_time_hours = 1.5
filter_space = pl.col("lenght_km") < filter_lenght_km
filter_time = pl.col("time_hours") < filter_time_hours



PyMC3 not installed


# Fit Paper: Do Here (single day separately)

In [2]:
# Feature Distribution Analysis with Class Conditioning
# ===================================================
# 
# This script performs comprehensive statistical analysis of trajectory features 
# (time_hours and lenght_km) conditioned on movement classes (0, 1, 2, 3).
# It fits exponential and power-law distributions to understand mobility patterns.
#
# INPUT DATA:
# -----------
# - FCM files: CSV files containing trajectory data with columns:
#   * time_hours: Duration of trajectories in hours
#   * lenght_km: Length of trajectories in kilometers  
#   * class: Movement class labels (0, 1, 2, 3)
# - Date range: 8 specific dates from 2022-2023
# - File format: bologna_mdt_{date}_{date}_fcm.csv
#
# CONFIGURATION PARAMETERS:
# -------------------------
# filter_lenght_km = 10      # Max trajectory length (km)
# filter_time_hours = 1.5    # Max trajectory duration (hours)
# bin_size_time_hours = 0.1  # Time histogram bin size
# bin_size_length_km = 1     # Length histogram bin size
# cut_thresholds = {1: 3, 2: 4}  # Length cutoffs for classes 1,2
#
# PROCESSING WORKFLOW:
# -------------------
# 1. DATA LOADING & FILTERING
#    - Load FCM data for each date
#    - Apply spatial filter: length < 10 km
#    - Apply temporal filter: time < 1.5 hours
#
# 2. FEATURE PROCESSING (Per Date)
#    For each feature (time_hours, lenght_km):
#    A. Overall Distribution (no class conditioning):
#       - Extract all feature values
#       - Compute histogram distribution
#       - Fit exponential/power-law models
#       - Calculate mean values
#    
#    B. Class-Conditional Distributions:
#       For each class (0, 1, 2, 3):
#       - Filter data by class label
#       - Compute class-specific histograms
#       - Apply class-specific length cuts (classes 1,2)
#       - Fit statistical models
#       - Calculate class means and variances
#
# 3. VISUALIZATION (Per Date)
#    - Plot distributions for all 4 classes on same figure
#    - Add fitted curves (exponential/power-law)
#    - Include inset plot showing class means
#    - Save as: {feature}_distribution_by_classes_{date}.png
#
# 4. CROSS-DATE AGGREGATION
#    - Normalize distributions across all dates
#    - Compute weighted averages per class
#    - Apply enrichment to standardize vector lengths
#    - Refit models to aggregated data
#
# 5. FINAL VISUALIZATION (Aggregated)
#    - Plot average distributions across all dates
#    - Show fitted models for each class
#    - Include error bars for class means
#    - Save as: {feature}_distribution_by_classes_averaged_over_days_concat.png
#
# OUTPUT FILES:
# -------------
# PLOTS:
# - Daily plots (16 files): {feature}_distribution_by_classes_{date}.png
#   Individual date analysis for time_hours and lenght_km
# - Aggregated plots (2 files): {feature}_distribution_by_classes_averaged_over_days_concat.png
#   Cross-date averaged results
#
# DATA FILES:
# - Fit parameters (4 CSV files):
#   * fit_info_final_version_{feature}.csv: Daily fit results
#   * fit_info_final_version_{feature}_concat.csv: Aggregated fit results
# - Contains: Model parameters, R² values, exponential/power-law coefficients
#
# KEY ANALYSIS FEATURES:
# ----------------------
# STATISTICAL MODELING:
# - Exponential fits: For length distributions
# - Power-law fits: For time distributions  
# - Model comparison: Automatic selection based on error metrics
#
# CLASS-SPECIFIC PROCESSING:
# - Dynamic thresholds: Different length cuts per class
# - Separate fitting: Independent models per movement class
# - Comparative analysis: Cross-class statistical comparison
#
# ROBUSTNESS:
# - Data enrichment: Standardized vector lengths for comparison
# - Normalization: Consistent probability distributions
# - Error handling: Graceful handling of missing/invalid data
#
# SCIENTIFIC APPLICATION:
# This analysis reveals how different mobility classes (slow/fast, short/long trips) 
# follow distinct statistical patterns, enabling:
# - Urban mobility modeling
# - Transportation pattern classification
# - Predictive trajectory analysis
# - Policy-relevant mobility insights

In [ ]:
import os
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


# Fcm,df_traj_cut column names
col_user_fcm = "id_act"
col_user_cut = "id_act"


# Data distribution parameters
bin_size_time_hours = 0.1
bin_size_length_km = 1
range_time_hours = [0.1, filter_time_hours]
range_length_km = [0.1, filter_lenght_km]
enriched_vector_length = 50

# Class-specific cut thresholds
cut_thresholds = {1: 3, 2: 4}

# Initialize data structures
data_structures = {
    "by_feature_day_class": init_data_structures(StrDates, Classes, Features),
    "by_feature_day": init_data_structures(StrDates, None, Features),
    "by_feature_class": init_data_structures(None, Classes, Features)
}

# Initialize fit information structures
fit_info, fit_info_concat, dict_Lkclass, dict_Lkclass_concat = init_fit_info()

print("Begin conditional class Plot distribution length and time")

# Create helper function to load data for a specific date
#def load_data(date_str):
#    path = os.path.join(fcm_dir, f"bologna_mdt_{date_str}_{date_str}_fcm.csv")
#    return pl.read_csv(path).filter(filter_space, filter_time)

# Parameters for processing
processing_params = {
    "range_time_hours": range_time_hours,
    "bin_size_time_hours": bin_size_time_hours,
    "range_length_km": range_length_km,
    "bin_size_length_km": bin_size_length_km,
    "enriched_vector_length": enriched_vector_length,
    "cut_thresholds": cut_thresholds
}

# Process each feature
for feature in Features:
    print(f"Feature: {feature}")
    
    # Reset fit info for each feature
    fit_info, fit_info_concat, dict_Lkclass, dict_Lkclass_concat = init_fit_info()
    all_data_combined = None
    
    # Process each date
    for date_str in StrDates:
        print(f"StrDate: {date_str}")
        
        # Load data for this date
        fcm_data = pl.read_csv(os.path.join(fcm_dir, f"{prefix_files}_{date_str}_{date_str}_fcm.csv")).filter(filter_space, filter_time)
        # df traj to cut
        df_cut_traj = pl.read_csv(os.path.join(fcm_dir, f"{prefix_files}_{date_str}_{date_str}_cut_traj.csv"))
        if col_user_cut in df_cut_traj:
            pass
        else:
            df_cut_traj = pl.read_csv(os.path.join(fcm_dir, f"{prefix_files}_{date_str}_{date_str}_cut_traj.csv"),separator=";")
        print(fcm_data)
        fcm_data = filter_users_if_one_point_out_of_polygon(fcm_data,df_cut_traj,col_user_fcm,col_user_cut)
        # Combine data for concatenated analysis
        if all_data_combined is None:
            all_data_combined = fcm_data
        else:
            all_data_combined = pl.concat([all_data_combined, fcm_data])
        
        # Process non-class specific data (entire dataset)
        feature_data = fcm_data[feature].to_list()
        x, n, A_fit, b_fit, y_fit, is_exp, x_mean = process_single_fit(
            feature_data, 
            feature,
            range_time_hours,
            bin_size_time_hours,
            range_length_km,
            bin_size_length_km,
            enriched_vector_length
        )
        
        # Update fit info
        fit_info_concat, dict_Lkclass_concat = update_fit_info(
            fit_info_concat,
            dict_Lkclass_concat,
            100,  # Special class for overall data
            x_mean,
            b_fit,
            is_exp,
            date_str
        )
        
        print(f"size x: {len(x)}, size n: {len(n)}, size y_fit: {len(y_fit)}, <x>: {x_mean}")
        
        # Store results in data structures
        data_structures["by_feature_day"]["x"][feature][date_str] = x
        data_structures["by_feature_day"]["y"][feature][date_str] = n
        data_structures["by_feature_day"]["y_fit"][feature][date_str] = y_fit
        data_structures["by_feature_day"]["y_avg"][feature][date_str] = x_mean
        
        # Process class-specific data
        x_means_day = np.zeros(len(Classes))
        x_variance_day = np.zeros(len(Classes))
        
        # Process each class
        for class_idx in Classes:
            print(f"Class: {class_idx}")
            
            # Filter data for this class
            class_data = fcm_data.filter(pl.col("class") == class_idx)[feature].to_list()
            
            # Process data with class-specific settings
            class_params = processing_params.copy()
            class_params["class_idx"] = class_idx
            
            x, n, A_fit, b_fit, y_fit, is_exp, x_mean = process_single_fit(
                class_data, 
                feature,
                **class_params
            )
            
            # Update fit info
            fit_info, dict_Lkclass = update_fit_info(
                fit_info,
                dict_Lkclass,
                class_idx,
                x_mean,
                b_fit,
                is_exp,
                date_str
            )
            
            # Store results in data structures
            data_structures["by_feature_day_class"]["x"][feature][date_str][class_idx] = x
            data_structures["by_feature_day_class"]["y"][feature][date_str][class_idx] = n
            data_structures["by_feature_day_class"]["y_fit"][feature][date_str][class_idx] = y_fit
            data_structures["by_feature_day_class"]["y_avg"][feature][date_str][class_idx] = x_mean
            
            # Calculate and store statistics
            x_means_day[class_idx] = x_mean
            x_variance_day[class_idx] = np.nanstd(class_data)
        
        # Create visualization for this day and feature
        print(f"Plot the distribution of the feature conditioned on the class {date_str}")
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        
        # Define x-limits based on feature
        x_lim_min = range_time_hours[0] if feature == "time_hours" else range_length_km[0]
        x_lim_max = range_time_hours[1] if feature == "time_hours" else range_length_km[1]
        
        # Plot each class
        for class_idx in Classes:
            ax = plot_distribution(
                ax,
                class_idx,
                data_structures["by_feature_day_class"]["x"][feature][date_str][class_idx],
                data_structures["by_feature_day_class"]["y"][feature][date_str][class_idx],
                data_structures["by_feature_day_class"]["y_fit"][feature][date_str][class_idx],
                labels,
                feature,
                x_lim_min,
                x_lim_max
            )
        
        ax.legend()
        
        # Add inset plot showing mean values
        ax_inset = add_inset_plot(
            fig,
            ax,
            Classes,
            {feature: {date_str: x_means_day}},
            [date_str],
            feature
        )
        
        # Save figure
        plt.savefig(os.path.join(PlotDir, f"{feature}_distribution_by_classes_{date_str}.png"))
        plt.close(fig)
    
    # Save fit information to CSV
    pl.DataFrame(fit_info).write_csv(os.path.join(PlotDir, f"fit_info_final_version_{feature}.csv"))
    
    # Process concatenated data across dates for each class
    print("Processing concatenated data")
    
    # Use last date's class 0 x-values as reference
    ref_x = np.array(data_structures["by_feature_day_class"]["x"][feature][StrDates[-1]][0])
    size_x = len(ref_x)
    
    # Initialize arrays for concatenated results
    x_means_concat = np.zeros(len(Classes))
    x_variance_concat = np.zeros(len(Classes))
    
    # Initialize concatenated y values
    for class_idx in Classes:
        data_structures["by_feature_class"]["y"][feature][class_idx] = np.zeros(size_x)
    
    # First, aggregate normalized distributions across dates
    for date_str in StrDates:
        for class_idx in Classes:
            # Get distribution for this class/date and normalize
            n = np.array(data_structures["by_feature_day_class"]["y"][feature][date_str][class_idx])
            n_normalized = n / np.sum(n) if np.sum(n) > 0 else n
            
            # Ensure vector length matches
            n_normalized = enrich_vector_to_length(n_normalized, size_x)
            
            # Add to aggregated distribution with equal weight per day
            data_structures["by_feature_class"]["y"][feature][class_idx] += (n_normalized / len(StrDates))
            
    # Process aggregated distributions
    for class_idx in Classes:
        # Get aggregated distribution
        y_agg = data_structures["by_feature_class"]["y"][feature][class_idx]
        
        # Apply class-specific cuts if needed
        if "len" in feature:
            if class_idx == 1:
                y_class = y_agg[ref_x > cut_thresholds[1]]
                x_fit = ref_x[ref_x > cut_thresholds[1]]
            elif class_idx == 2:
                y_class = y_agg[ref_x > cut_thresholds[2]]
                x_fit = ref_x[ref_x > cut_thresholds[2]]
            else:
                y_class = y_agg
                x_fit = ref_x
        else:
            y_class = y_agg
            x_fit = ref_x
        
        # Fit the distribution
        x_result, n_result, A_fit, b_fit, y_fit, is_exp = compute_single_fit_from_xy(
            x_fit,
            y_class,
            feature,
            bin_size_time_hours,
            bin_size_length_km,
            range_time_hours,
            range_length_km,
            enriched_vector_length,
            class_idx
        )
        
        # Normalize
        n_result = n_result / np.sum(n_result) if np.sum(n_result) > 0 else n_result
        
        # Calculate statistics
        x_mean = np.sum(n_result * ref_x) if len(n_result) == len(ref_x) else np.mean(n_result * x_result)
        x_variance = np.sqrt(np.sum(n_result * (ref_x - x_mean)**2) / len(n_result)) if len(n_result) == len(ref_x) else 0
        
        # Update fit info
        fit_info_concat, dict_Lkclass_concat = update_fit_info(
            fit_info_concat,
            dict_Lkclass_concat,
            class_idx,
            x_mean,
            b_fit,
            is_exp,
            "concatenated"  # Use special date identifier for concatenated data
        )
        
        # Store results
        data_structures["by_feature_class"]["x"][feature][class_idx] = ref_x
        data_structures["by_feature_class"]["y"][feature][class_idx] = n_result
        data_structures["by_feature_class"]["y_fit"][feature][class_idx] = y_fit
        data_structures["by_feature_class"]["y_avg"][feature][class_idx] = x_mean
        
        # Store statistics
        x_means_concat[class_idx] = x_mean
        x_variance_concat[class_idx] = x_variance
    
    # Plot concatenated distributions
    print("Plot the distribution of the feature conditioned on the class (concatenated)")
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    # Plot each class
    for class_idx in Classes:
        ax = plot_distribution(
            ax,
            class_idx,
            data_structures["by_feature_class"]["x"][feature][class_idx],
            data_structures["by_feature_class"]["y"][feature][class_idx],
            data_structures["by_feature_class"]["y_fit"][feature][class_idx],
            labels,
            feature,
            x_lim_min,
            x_lim_max
        )
    
    ax.legend()
    
    # Add inset plot
    ax_inset = inset_axes(ax, width="20%", height="20%", loc="upper center")
    ax_inset.errorbar(
        Classes,
        x_means_concat,
        yerr=x_variance_concat,
        fmt='o',
        color='blue',
        ecolor='red',
        elinewidth=1,
        capsize=3,
    )
    
    ax_inset.set_xticks(Classes)
    if feature == "time_hours":
        ax_inset.set_xlabel("class", fontsize=12)
        ax_inset.set_ylabel(r"$\langle t \rangle$ (h)", fontsize=12)
    else:
        ax_inset.set_xlabel("class", fontsize=12)
        ax_inset.set_ylabel(r"$\langle l \rangle$ (km)", fontsize=12)
    
    # Save figure for concatenated data
    plt.savefig(os.path.join(PlotDir, f"{feature}_distribution_by_classes_averaged_over_days_concat.png"))
    plt.close(fig)
    
    # Save fit information to CSV
    pl.DataFrame(fit_info_concat).write_csv(os.path.join(PlotDir, f"fit_info_final_version_{feature}_concat.csv"))

Begin conditional class Plot distribution length and time
Feature: time_hours
StrDate: 2022-01-31
shape: (27_834, 21)
┌────────────┬─────────┬──────┬──────────┬───┬───────────┬───────────┬────────────┬───────────┐
│ id_act     ┆ lenght  ┆ time ┆ av_speed ┆ … ┆ speed_kmh ┆ lenght_km ┆ time_hours ┆ class_new │
│ ---        ┆ ---     ┆ ---  ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---        ┆ ---       │
│ i64        ┆ f64     ┆ i64  ┆ f64      ┆   ┆ f64       ┆ f64       ┆ f64        ┆ i64       │
╞════════════╪═════════╪══════╪══════════╪═══╪═══════════╪═══════════╪════════════╪═══════════╡
│ 3221230664 ┆ 1306.23 ┆ 389  ┆ 9.70331  ┆ … ┆ 34.931916 ┆ 1.30623   ┆ 0.108056   ┆ 2         │
│ 3221296375 ┆ 2538.69 ┆ 674  ┆ 13.6348  ┆ … ┆ 49.08528  ┆ 2.53869   ┆ 0.187222   ┆ 2         │
│ 3221352640 ┆ 595.956 ┆ 541  ┆ 1.08706  ┆ … ┆ 3.913416  ┆ 0.595956  ┆ 0.150278   ┆ 2         │
│ 3221361764 ┆ 1132.24 ┆ 184  ┆ 6.55899  ┆ … ┆ 23.612364 ┆ 1.13224   ┆ 0.051111   ┆ 2         │
│ 3221378226 ┆ 725

# (Averaged all days): You must run it after the pipeline

In [4]:
# Average Distribution Analysis
# ============================
#
# Computes average distributions across all dates for each feature,
# fits appropriate statistical models, and creates publication-ready plots.
#
# KEY FEATURES:
# - Cross-date averaging with proper normalization
# - Forced model selection: Power law for time, Exponential for length
# - Smoothing with moving average windows
# - Plotting with logarithmic scales
#
# OUTPUT:
# - {feature}_average_distribution{case}.png
# - average_distribution_fit_params{case}.json

In [ ]:
config = {
    'features': ["time_hours", "lenght_km"],
    'dates': ["2022-01-31", "2022-07-01", "2022-08-05", "2022-11-11", 
             "2022-12-30", "2023-01-01", "2022-12-31", "2023-03-18"],
    'fcm_dir': fcm_dir,
    'plot_dir': PlotDir,
    
    'filter_params': {
        'filter_length_km': 10,
        'filter_time_hours': 1.5,
        'filter_space': pl.col("lenght_km") < 10,
        'filter_time': pl.col("time_hours") < 1.5,
    },
    
    'distribution_params': {
        'bin_size_time_hours': 0.05,
        'bin_size_length_km': 1,
        'range_time_hours': [0.1, 1.5],
        'range_length_km': [0.1, 10],
        'enriched_vector_length': 50,
        'cut_length': 4,
        'window_size': 3,
        'case': "_forced_fits"  # Add a suffix to distinguish these from other plots
    }
}
run_analysis(config)

Starting time and length distribution analysis...
Feature: time_hours
Date: 2022-01-31
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.2987
Date: 2022-07-01
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.3071
Date: 2022-08-05
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.2547
Date: 2022-11-11
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.3441
Date: 2022-12-30
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.3224
Date: 2023-01-01
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.3341
Date: 2022-12-31
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.3435
Date: 2023-03-18
bin_size:  0.05
size x: 50, size n: 50, size y_fit: 50, <x>: 0.3428
Feature: lenght_km
Date: 2022-01-31
bin_size:  1
size x: 50, size n: 50, size y_fit: 50, <x>: 3.1570
Date: 2022-07-01
bin_size:  1
size x: 50, size n: 50, size y_fit: 50, <x>: 3.3623
Date: 2022-08-05
bin_size:  1
size x: 50, size n: 50, size y_fit: 5

{'time_hours': {'expo': None, 'pl': -1.310069353363117},
 'lenght_km': {'expo': -0.2642825531298348, 'pl': None}}

# Plot all days with fit no-class partition

In [6]:
def calculate_and_plot_average_distributions(
    features: List[str],
    dates: List[str],
    fcm_dir: str,
    plot_dir: str,
    filter_space: pl.Expr,
    filter_time: pl.Expr,
    bin_size_time_hours: float,
    bin_size_length_km: float,
    range_time_hours: List[float],
    range_length_km: List[float],
    enriched_vector_length: int = 50,
    cut_length: float = 4,
    window_size: int = 3,
    case: str = ""
) -> Dict[str, Dict]:
    """
    Calculate the average distribution over all days for each feature, fit appropriate models, and plot.
    """
    # Create output directory if it doesn't exist
    os.makedirs(plot_dir, exist_ok=True)
    
    # Initialize dictionaries to store data
    daily_distributions = {feature: {} for feature in features}
    feature_means = {feature: {} for feature in features}
    
    # Initialize result dictionaries for fits
    fit_params = {feature: {} for feature in features}
    
    # Step 1: Extract data and compute distribution for each day and feature
    for feature in features:
        print(f"Processing feature: {feature}")
        
        # Step 1a: For each day, extract data and compute distribution
        for date_str in dates:
            print(f"  Processing date: {date_str}")
            
            # Load data for this day
            fcm_data = load_data(fcm_dir, date_str, filter_space, filter_time)
            if fcm_data is None or fcm_data.height == 0:
                print(f"  No data for {date_str}, skipping")
                continue
                
            # Extract feature data
            feature_data = fcm_data[feature].to_numpy()
            if len(feature_data) == 0:
                print(f"  No {feature} data for {date_str}, skipping")
                continue
            
            # Compute mean for this day
            feature_means[feature][date_str] = np.nanmean(feature_data)
            
            # Compute distribution for this day
            try:
                bins, bin_range = from_feature_to_nin_bin_range(
                    feature, 
                    range_time_hours, 
                    bin_size_time_hours, 
                    range_length_km, 
                    bin_size_length_km
                )
                
                x, n = from_data_to_cut_distribution(feature_data, bins, bin_range)
                
                # Store the distribution for this day
                daily_distributions[feature][date_str] = {
                    'x': x,
                    'n': n
                }
                print(f"  Distribution for {feature} on {date_str}: x shape = {len(x)}, n shape = {len(n)}")
            except Exception as e:
                print(f"  Error computing distribution for {date_str}, feature {feature}: {e}")
    
    # Step 2: Compute average distribution over all days for each feature
    average_distributions = {}
    for feature in features:
        print(f"Computing average distribution for {feature}")
        
        # Create a reference x-axis for this feature with consistent points
        if feature == 'time_hours':
            ref_x = np.linspace(range_time_hours[0], range_time_hours[1], enriched_vector_length)
        else:  # lenght_km
            ref_x = np.linspace(range_length_km[0], range_length_km[1], enriched_vector_length)
        
        # Initialize arrays for summing distributions
        summed_n = np.zeros(enriched_vector_length)
        valid_days = 0
        
        # Process each day
        for date_str, dist in daily_distributions[feature].items():
            if len(dist['x']) == 0 or len(dist['n']) == 0:
                print(f"  Empty distribution for {feature} on {date_str}, skipping")
                continue
                
            # Check if x and n have the same length
            if len(dist['x']) != len(dist['n']):
                print(f"  Warning: x and n have different lengths for {feature} on {date_str}")
                max_len = max(len(dist['x']), len(dist['n']))
                print(f"  enrich {max_len}")
                x_day = enrich_vector_to_length(dist['x'], max_len)
                n_day = enrich_vector_to_length(dist['n'], max_len)
            else:
                x_day = dist['x']
                n_day = dist['n']
            
            # Handle case where x values might not be sorted (required for np.interp)
            sort_idx = np.argsort(x_day)
            x_day = np.array(x_day)[sort_idx]
            n_day = np.array(n_day)[sort_idx]
            
            # Resample the distribution to match ref_x
            try:
                resampled_n = np.interp(
                    ref_x, 
                    x_day, 
                    n_day,
                    left=0,
                    right=0
                )
                
                # Normalize the distribution
                if np.sum(resampled_n) > 0:
                    resampled_n = resampled_n / np.sum(resampled_n)
                
                # Add to sum
                summed_n += resampled_n
                valid_days += 1
            except Exception as e:
                print(f"  Error resampling distribution for {feature} on {date_str}: {e}")
                print(f"  x_day shape: {x_day.shape}, n_day shape: {n_day.shape}")
        
        if valid_days == 0:
            print(f"  No valid distributions for {feature}")
            continue
            
        # Compute average
        avg_n = summed_n / valid_days
        
        # Smooth the distribution with a moving average
        avg_n_smooth = np.zeros_like(avg_n)
        for i in range(len(avg_n)):
            start = max(0, i - window_size // 2)
            end = min(len(avg_n), i + window_size // 2 + 1)
            avg_n_smooth[i] = np.mean(avg_n[start:end])
            
        # Store average distribution
        average_distributions[feature] = {
            'x': ref_x,
            'n': avg_n_smooth,
            'mean': np.mean([mean for mean in feature_means[feature].values()])
        }
    
    # Step 3: Fit models to the average distributions
    for feature in features:
        if feature not in average_distributions:
            continue
            
        x = average_distributions[feature]['x']
        n = average_distributions[feature]['n']
        
        if feature == 'time_hours':
            # Fit power law for time_hours
            try:
                # Workaround for zeros in power law fitting
                # Find non-zero values for proper power law fitting
                mask_nonzero = np.logical_and(x > 0, n > 0)
                if np.sum(mask_nonzero) < 3:  # Need at least 3 points for a decent fit
                    print(f"  Not enough non-zero data points for power law fit of {feature}")
                    # Apply small epsilon to zeros
                    x_pl = x.copy()
                    n_pl = n.copy()
                    n_pl[n_pl <= 0] = np.min(n_pl[n_pl > 0]) * 0.1  # Small fraction of smallest non-zero value
                else:
                    x_pl = x[mask_nonzero]
                    n_pl = n[mask_nonzero]
                
                # Fit both models
                A_exp, beta_exp, exp_, error_exp, R2_exp, A_pl, alpha_pl, pl_, error_pl, R2_pl, bins_plot = \
                    compare_exponential_power_law_from_xy(x_pl, n_pl)
                
                # For time_hours, always use power law regardless of error
                # But extend the fit to the full range
                full_pl = A_pl * np.power(x, alpha_pl)  # Calculate over the full range
                
                fit_params[feature] = {
                    'model': 'power_law',
                    'A': A_pl,
                    'alpha': alpha_pl,
                    'R2': R2_pl,
                    'fitted_y': full_pl,
                    'fitted_range': x_pl  # Store the range that was actually used for fitting
                }
                print(f"  Fit power law to {feature}: alpha = {alpha_pl:.4f}, R² = {R2_pl:.4f}")
                
            except Exception as e:
                print(f"  Error fitting power law to {feature}: {e}")
                fit_params[feature] = {
                    'model': 'none',
                    'error': str(e)
                }
        
        elif 'lenght' in feature:
            # Fit exponential only up to cut_length for lenght_km
            try:
                # Apply cut length
                mask = x <= cut_length
                x_masked = x[mask]
                n_masked = n[mask]
                
                if len(x_masked) == 0:
                    print(f"  No data points below cut_length={cut_length} for {feature}")
                    continue
                    
                # Fit exponential up to cut length
                A_exp, beta_exp, exp_, error_exp, R2_exp, bins_plot = fit_expo(x_masked, n_masked)
                
                # Create full-range fitted curve for plotting (but fit is only valid up to cut_length)
                full_exp = A_exp * np.exp(beta_exp * x)
                
                fit_params[feature] = {
                    'model': 'exponential',
                    'A': A_exp,
                    'beta': beta_exp,
                    'R2': R2_exp,
                    'fitted_y': full_exp,
                    'cut_length': cut_length
                }
                print(f"  Fit exponential to {feature}: beta = {beta_exp:.4f}, R² = {R2_exp:.4f}")
                
            except Exception as e:
                print(f"  Error fitting exponential to {feature}: {e}")
                fit_params[feature] = {
                    'model': 'none',
                    'error': str(e)
                }
    
    # Step 4: Plot the average distributions with fits
    for feature, dist in average_distributions.items():
        print(f"Plotting {feature} distribution")
        
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Plot the average distribution
        ax.scatter(dist['x'], dist['n'], 
                  label=f"Average Distribution", 
                  alpha=0.7, color='black', s=30)
        main_labels = ["average"]
        # Plot individual day distributions with lower opacity
        for date_str, day_dist in daily_distributions[feature].items():
            if len(day_dist['x']) == 0 or len(day_dist['n']) == 0:
                continue
            main_labels.append(date_str)
    
            # Check if x and n have the same length
            if len(day_dist['x']) != len(day_dist['n']):
                max_len = max(len(day_dist['x']), len(day_dist['n']))
                x_day = enrich_vector_to_length(day_dist['x'],max_len)
                n_day = enrich_vector_to_length(day_dist['n'],max_len)
            else:
                x_day = day_dist['x']
                n_day = day_dist['n']
            
            # Handle case where x values might not be sorted (required for np.interp)
            sort_idx = np.argsort(x_day)
            x_day = np.array(x_day)[sort_idx]
            n_day = np.array(n_day)[sort_idx]
            
            try:
                # Resample to match ref_x
                resampled_n = np.interp(
                    dist['x'], 
                    x_day, 
                    n_day,
                    left=0,
                    right=0
                )
                
                # Normalize
                if np.sum(resampled_n) > 0:
                    resampled_n = resampled_n / np.sum(resampled_n)
                
                ax.scatter(dist['x'], resampled_n, 
                          label=date_str, 
                          alpha=0.3, s=10)
            except Exception as e:
                print(f"  Error plotting day distribution for {feature} on {date_str}: {e}")
        
        # Plot the fit if available
        if feature in fit_params and fit_params[feature].get('model') != 'none':
            fit_info = fit_params[feature]
            
            if fit_info['model'] == 'power_law':
                # Normalize for plotting
                y_fit = fit_info['fitted_y']
                if np.sum(y_fit) > 0:
                    y_fit = y_fit / np.sum(y_fit)
                
                # Highlight the actual fitted range
                ax.plot(dist['x'], y_fit, '--', 
                       linewidth=2, color='blue',
                       label=f"Power Law Fit (α = {fit_info['alpha']:.4f})")
                
                # Add annotation about power law fit if we had to handle zeros
                if 'fitted_range' in fit_info and len(fit_info['fitted_range']) < len(dist['x']):
                    ax.text(0.05, 0.05, "Note: Power law fit excludes zero values", 
                           transform=ax.transAxes, fontsize=10, alpha=0.7)
                
            elif fit_info['model'] == 'exponential':
                # Normalize for plotting
                y_fit = fit_info['fitted_y']
                if np.sum(y_fit) > 0:
                    y_fit = y_fit / np.sum(y_fit)
                
                # Get the index corresponding to the cut length
                cut_idx = np.searchsorted(dist['x'], cut_length)
                
                # Plot the fit differently before and after the cut length
                ax.plot(dist['x'][:cut_idx+1], y_fit[:cut_idx+1], '-', 
                       linewidth=3, color='red',
                       label=f"Exponential Fit (β = {fit_info['beta']:.4f})")
                # Add vertical line at cut_length
                ax.axvline(x=cut_length, color='gray', linestyle='--', alpha=0.7)
                ax.text(cut_length*1.05, ax.get_ylim()[1]*0.8, f'cut = {cut_length} km', 
                       rotation=90, color='gray')
        
        # Configure plot
        ax.set_yscale('log')
        
        if feature == 'time_hours':
            ax.set_xlabel('t(h)', fontsize=14)
            ax.set_ylabel('P(t)', fontsize=14)
            ax.set_xlim(range_time_hours[0], range_time_hours[1])
        else:
            ax.set_xlabel('l(km)', fontsize=14)
            ax.set_ylabel('P(l)', fontsize=14)
            ax.set_xlim(range_length_km[0], range_length_km[1])
        ax.set_yscale('log')

        # Get the current data range
        ymin, ymax = ax.get_ylim()

        # Create exactly 3 logarithmically spaced ticks
        log_ymin, log_ymax = np.log10(ymin), np.log10(ymax)
        log_ticks = np.linspace(log_ymin, log_ymax, 3)  # 3 points evenly spaced in log space
        tick_positions = 10**log_ticks

        # Set these ticks directly
        ax.set_yticks(tick_positions)

        # Create a formatter that displays the exponent
        def log_formatter(y, pos):
            exponent = np.log10(y)
            # Round to 1 decimal place if not close to an integer
            if abs(exponent - round(exponent)) < 0.05:
                return f'$10^{{{int(round(exponent))}}}$'
            else:
                return f'$10^{{{exponent:.1f}}}$'

        ax.yaxis.set_major_formatter(ticker.FuncFormatter(log_formatter))

        # Add minor ticks between the major ticks for better visualization
        minor_locator = ticker.LogLocator(base=10.0, subs=np.arange(2, 10) * 0.1)
        ax.yaxis.set_minor_locator(minor_locator)        # Add inset with mean values
        
        # Keep only the first few legend items to avoid overcrowding
        ax.legend(main_labels, 
                 loc='upper right', bbox_to_anchor=(0.98, 0.98),
                 fontsize=12, framealpha=0.8)
        
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dir, f"{feature}_average_distribution{case}.png"), dpi=300)
        plt.close(fig)
    
    # Save fit parameters
    fit_params_for_json = {
        feature: {
            'model': params.get('model', 'none'),
            'parameters': {
                'A': float(params.get('A', 0)),
                'alpha' if params.get('model') == 'power_law' else 'beta': 
                    float(params.get('alpha' if params.get('model') == 'power_law' else 'beta', 0))
            },
            'R2': float(params.get('R2', 0)),
            'cut_length': float(params.get('cut_length', 0)) if params.get('model') == 'exponential' else None
        }
        for feature, params in fit_params.items()
        if params.get('model') != 'none'
    }
    
    try:
        with open(os.path.join(plot_dir, f"average_distribution_fit_params{case}.json"), "w") as f:
            json.dump(fit_params_for_json, f, indent=4)
    except Exception as e:
        print(f"Error saving fit parameters: {e}")
    
    return fit_params

In [7]:
def run_average_distribution_analysis(config: Dict[str, Any]) -> Dict[str, Dict]:
    """
    Run the average distribution analysis pipeline.
    
    Args:
        config: Dictionary containing configuration parameters
            - features: List of features to analyze
            - dates: List of date strings
            - fcm_dir: Directory containing FCM data files
            - plot_dir: Directory to save plots
            - filter_params: Dictionary of filter parameters
            - distribution_params: Dictionary of distribution parameters
        
    Returns:
        Dictionary of fit parameters for each feature
    """
    # Extract configuration parameters
    features = config.get('features', ["time_hours", "lenght_km"])
    dates = config.get('dates', [])
    fcm_dir = config.get('fcm_dir', '')
    plot_dir = config.get('plot_dir', '')
    
    # Filter parameters
    filter_params = config.get('filter_params', {})
    filter_length_km = filter_params.get('filter_length_km', 10)
    filter_time_hours = filter_params.get('filter_time_hours', 1.5)
    filter_space = filter_params.get('filter_space', pl.col("lenght_km") < filter_length_km)
    filter_time = filter_params.get('filter_time', pl.col("time_hours") < filter_time_hours)
    
    # Distribution parameters
    dist_params = config.get('distribution_params', {})
    bin_size_time_hours = dist_params.get('bin_size_time_hours', 0.05)
    bin_size_length_km = dist_params.get('bin_size_length_km', 1)
    range_time_hours = dist_params.get('range_time_hours', [0.1, filter_time_hours])
    range_length_km = dist_params.get('range_length_km', [0.1, filter_length_km])
    enriched_vector_length = dist_params.get('enriched_vector_length', 50)
    cut_length = dist_params.get('cut_length', 4)
    window_size = dist_params.get('window_size', 3)
    case = dist_params.get('case', "")
    
    print("Starting average distribution analysis...")
    
    # Calculate and plot average distributions
    fit_params = calculate_and_plot_average_distributions(
        features,
        dates,
        fcm_dir,
        plot_dir,
        filter_space,
        filter_time,
        bin_size_time_hours,
        bin_size_length_km,
        range_time_hours,
        range_length_km,
        enriched_vector_length,
        cut_length,
        window_size,
        case
    )
    
    print("Analysis complete. Results saved to:", plot_dir)
    
    return fit_params

In [ ]:
config = {
    'features': ["time_hours", "lenght_km"],
    'dates': ["2022-01-31", "2022-07-01", "2022-08-05", "2022-11-11", 
             "2022-12-30", "2023-01-01", "2022-12-31", "2023-03-18"],
    'fcm_dir': fcm_dir,
    'plot_dir': PlotDir,
    
    'filter_params': {
        'filter_length_km': 10,
        'filter_time_hours': 1.5,
        'filter_space': pl.col("lenght_km") < 10,
        'filter_time': pl.col("time_hours") < 1.5,
    },
    
    'distribution_params': {
        'bin_size_time_hours': 0.05,
        'bin_size_length_km': 1,
        'range_time_hours': [0.1, 1.5],
        'range_length_km': [0.1, 10],
        'enriched_vector_length': 50,
        'cut_length': 4,
        'window_size': 3,
        'case': "_forced_fits"  # Add a suffix to distinguish these from other plots
    }
}

fit_params = run_average_distribution_analysis(config)

Starting average distribution analysis...
Processing feature: time_hours
  Processing date: 2022-01-31
bin_size:  0.05
  Distribution for time_hours on 2022-01-31: x shape = 28, n shape = 27
  Processing date: 2022-07-01
bin_size:  0.05
  Distribution for time_hours on 2022-07-01: x shape = 28, n shape = 27
  Processing date: 2022-08-05
bin_size:  0.05
  Distribution for time_hours on 2022-08-05: x shape = 28, n shape = 27
  Processing date: 2022-11-11
bin_size:  0.05
  Distribution for time_hours on 2022-11-11: x shape = 28, n shape = 27
  Processing date: 2022-12-30
bin_size:  0.05
  Distribution for time_hours on 2022-12-30: x shape = 28, n shape = 27
  Processing date: 2023-01-01
bin_size:  0.05
  Distribution for time_hours on 2023-01-01: x shape = 28, n shape = 27
  Processing date: 2022-12-31
bin_size:  0.05
  Distribution for time_hours on 2022-12-31: x shape = 28, n shape = 27
  Processing date: 2023-03-18
bin_size:  0.05
  Distribution for time_hours on 2023-03-18: x shape = 

# Fit Gauss for paper -> still experimental and will remain such since we force to avoid the fite over 40 km/h

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from PlotSettings import *
from analysisPlot import *
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False
# Define Gaussian function
def gaussian(x, A, mu, sigma):
    return A * np.exp(-0.5 * ((x - mu) / sigma)**2)


def gaussian_fit(x, y):
    """
    Fit a Gaussian function to x,y data points
    
    Parameters:
    -----------
    x : array-like
        x-coordinates
    y : array-like
        y-coordinates (observations)
        
    Returns:
    --------
    params : tuple
        (A, mu, sigma) parameters of the Gaussian fit
        A: amplitude
        mu: mean
        sigma: standard deviation
    covariance : 2D array
        Covariance matrix of the parameters
    fitted_y : array
        y values of the fitted Gaussian at x positions
    """
    # Initial guesses for parameters
    A_guess = np.max(y)
    mu_guess = np.sum(x * y) / np.sum(y)  # Center of mass
    sigma_guess = np.sqrt(np.sum(y * (x - mu_guess)**2) / np.sum(y))
    
    
    try:
        # Perform the fit
        params, covariance = curve_fit(
            gaussian, x, y, 
            p0=[A_guess, mu_guess, sigma_guess],
            maxfev=50000
        )
        
        # Calculate fitted y values
        fitted_y = gaussian(x, *params)
        
        return params, covariance, fitted_y
    
    except RuntimeError:
        print("Error: Fit did not converge")
        return (A_guess, mu_guess, sigma_guess), None, None

import json
key_2_int_class = {"1 slowest":0,
 "2 slowest":1,
 "middle velocity class":2,
 "1 quickest":3}
int_class_2_color = {0:"blue",
                    1:"orange",
                    2:"green",
                    3:"red"}
with open(os.path.join(PlotDir,"aggregated_average_speed_kmh_plots.json"),"r") as f:
    aggregated_average_speed = json.load(f)
    aggregated_average_speed = {key:{"x":v["x"],"y":v["y"]} for key,v in aggregated_average_speed.items()}
fig,ax = plt.subplots(1,1,figsize = (10,10))
for key in aggregated_average_speed.keys():
    int_class = key_2_int_class[key]
    # Upload the x,y
    x = aggregated_average_speed[key]["x"]
    y = aggregated_average_speed[key]["y"]
    Feature = "speed_kmh"
    Feature2Label = {"time_hours":"t","lenght_km":"L","speed_kmh":"v (km/h)"}
    # Scatter Points
    if int_class == 2:
        x_fit = np.array(x)[np.array(x) <= 40]
        y_fit = np.array(y)[np.array(x) <= 40]
        params, covariance, fitted_y = gaussian_fit(x_fit, y_fit)

    else:
        x_fit = np.array(x)
        y_fit = np.array(y)
        params, covariance, fitted_y = gaussian_fit(x_fit, y_fit)    
    ax.scatter(x,y,color = int_class_2_color[int_class])
    ax.plot(x,gaussian(x,params[0],params[1],params[2]),color = int_class_2_color[int_class])
    ax.set_xticks(np.arange(x[0],x[-1],20))
    ax.set_yticks(np.arange(min(y),max(y),0.05))
    ax.set_xlabel(Feature2Label[Feature])
    ax.set_ylabel("P(v)")
    ax.set_xlim(left = 0,right = 150)
#    ax.set_xscale(Feature2ScaleBins[Feature])
#    ax.set_yscale(Feature2ScaleCount[Feature])
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
ax_inset = inset_axes(ax, width="20%", height="20%")
PlotInsetSpeedAverage(Feature,ax_inset,PlotDir)
plt.savefig(os.path.join(PlotDir,"aggregated_average_speed_kmh_plots.png"))
